Project Ideas

- Google Maps Distance API - https://developers.google.com/maps/documentation/distance-matrix/intro
    - Set a marker A, set marker B, return the travel time
    - Set a marker A, calculate routes to multiple points on circle of radius X
    - Set a marker A, calculate routes to many points on a grid within circle of radius x
    
- House price API
    - Find average price at point A
    
- Display / Output
    - Load a google map
    - Display points A, B
    - Display a circle with many B points
    - Display a circle with many B points inside
    - Display a colour coded point by house price etc
    
    

# Super Basic Example!

In [330]:
import pandas as pd
import numpy as np
import requests
import json
import datetime
from urllib.request import urlretrieve
from urllib.parse import urlencode
import tqdm

#!pip install polyline

import polyline

API_KEY = open('API_key.txt','r').read()

Origin = 'Monzo, Finsbury Square'
Destination = 'Epsom'
travel_mode = 'transit'

google_url = 'https://maps.googleapis.com/maps/api/distancematrix/json?origins=' + Origin + '&destinations=' + Destination + '&key=' + API_KEY + '&mode=' +travel_mode 
google_request = requests.get(google_url)

google_json = google_request.json()
print(json.dumps(google_json, indent=2))

{
  "destination_addresses": [
    "Epsom, UK"
  ],
  "origin_addresses": [
    "38 Finsbury Square, Finsbury, London EC2A 1PX, UK"
  ],
  "rows": [
    {
      "elements": [
        {
          "distance": {
            "text": "29.0 km",
            "value": 29015
          },
          "duration": {
            "text": "58 mins",
            "value": 3473
          },
          "status": "OK"
        }
      ]
    }
  ],
  "status": "OK"
}


In [365]:

travel_mode = 'walking'
# driving, walking, bicycling, transit   are the possible options
from urllib.request import urlretrieve
from urllib.parse import urlencode


def distance_api(start_lat, start_lon, travel_mode, end_lat, end_lon, rebuild_table):
    
    global logging_table
    global Origin
    global Destination
    
    #read in API key from local directory
    API_KEY = open('API_key.txt','r').read()
    
    Origin = str(str(start_lat) + ', ' + str(start_lon))
    Destination = str(str(end_lat) + ', ' + str(end_lon))
    
    #convert coordinates to encoded polyline to make url easier to parse
    origin_polyline = 'enc:' + polyline.encode([(start_lat, start_lon)], 5) + ':'
    destination_polyline = 'enc:' + polyline.encode([(end_lat, end_lon)], 5) + ':'
    
    
    google_url = 'https://maps.googleapis.com/maps/api/distancematrix/json?origins=' + origin_polyline + '&destinations=' + destination_polyline + '&key=' + API_KEY + '&mode=' +travel_mode 
    
    google_request = requests.get(google_url)

    google_json = google_request.json()

    
    try:
        distance_text = google_json['rows'][0]['elements'][0]['distance']['text']

        
    except:
        distance_text = 'API ERROR'
        duration_text = 'API ERROR'
        duration_value = 0
        
    else:
        distance_text = google_json['rows'][0]['elements'][0]['distance']['text']
        duration_text = google_json['rows'][0]['elements'][0]['duration']['text']

        duration_value = google_json['rows'][0]['elements'][0]['duration']['value']

   # print(Origin + ' is ' + distance_text + ' away from ' + Destination +'. It will take ' + duration_text + ' to get there by ' + travel_mode)

    result_list = [datetime.datetime.now(),Origin,Destination,distance_text,duration_text,duration_value,travel_mode]

    #initialise logging table
    
    #print('rebuild_table = ' + str(rebuild_table))
    if rebuild_table == 1:
        del logging_table
    
    try:
        logging_table
    except NameError:
        #print('nameerror flagged')
        column_labels = ['runtime','Origin','Destination','distance','travel_time','duration_value','travel_mode']
        logging_table = pd.DataFrame(columns = column_labels)
        logging_table = logging_table.append(pd.Series(result_list, index = logging_table.columns), ignore_index=True)
    else:
        #print('no nameerror')
        logging_table = logging_table.append(pd.Series(result_list, index = logging_table.columns), ignore_index=True)
      
    return (logging_table)

In [239]:
#distance_api(start_lat= 51.520406,start_lon= -0.085969, travel_mode= 'transit', end_lat= 51.640425, end_lon= 0.4384, rebuild_table = True)

,runtime,Origin,Destination,distance,travel_time,duration_value,travel_mode
0,2020-03-04 14:56:22.312066,"51.520406, -0.085969","51.640425, 0.4384",42.7 km,1 hour 11 mins,4253,transit


In [368]:
import math

# Monzo Office Location
# start_lat = 51.520406
# start_lon = -0.085969


def circle_coords(start_lat,start_lon,distance,n_points):
    
    global coordinates_df
    
    R = 6378.1 #Radius of the Earth
 
    
    lat1 = math.radians(start_lat) #Current lat point converted to radians
    lon1 = math.radians(start_lon) #Current long point converted to radians
    
    # create a dataframe to populate
    column_labels = ['i','bearing','distance','start_lat','start_lon','end_lat','end_lon']
    coordinates_df = pd.DataFrame(columns = column_labels)
    
    for i in range(0, n_points):
                
        brng = math.radians(i * (360/n_points))    #convert bearing to radians
        lat1 = math.radians(start_lat) #Current lat point converted to radians
        lon1 = math.radians(start_lon) #Current long point converted to radians
        
        end_lat = math.degrees(math.asin( math.sin(lat1)*math.cos(distance/R) + math.cos(lat1)*math.sin(distance/R)*math.cos((brng))))
        end_lon = math.degrees(lon1 + math.atan2(math.sin(brng)*math.sin(distance/R)*math.cos(lat1), math.cos(distance/R)-math.sin(lat1)*math.sin(lat2)))
        
        coords_list = [i+1, i*(360/n_points), distance, start_lat, start_lon, end_lat, end_lon]
        
        coordinates_df = coordinates_df.append(pd.Series(coords_list, index = coordinates_df.columns), ignore_index=True)

    return (coordinates_df)


In [340]:

def concentric_circles(split,reset_table,start_latitude,start_longitude,number_points,distance):
   
    global coordinates_df_conc
    
    if reset_table == 1:
        coordinates_df_conc = 1
        del coordinates_df_conc  


    for i in range(1,distance+split):
        
        try:
            coordinates_df_conc
            
        except NameError:
            circle_coords(start_lat = start_latitude, start_lon = start_longitude, distance = i, n_points = number_points)
            coordinates_df_conc = coordinates_df
            
        else:

                
            circle_coords(start_lat = start_latitude, start_lon = start_longitude, distance = i, n_points = number_points)
            
            coordinates_df_conc = pd.concat([coordinates_df_conc,coordinates_df], ignore_index=True)
         
        
       
    
    return (coordinates_df_conc)



In [339]:
#concentric_circles(split=1,reset_table=1,start_latitude=51.520406, start_longitude=-0.085969, number_points=24, distance=10)

,i,bearing,distance,start_lat,start_lon,end_lat,end_lon
0,1.0,0.0,1.0,51.520406,-0.085969,51.529389,-0.085969
1,2.0,15.0,1.0,51.520406,-0.085969,51.529083,-0.080316
2,3.0,30.0,1.0,51.520406,-0.085969,51.528185,-0.075048
3,4.0,45.0,1.0,51.520406,-0.085969,51.526758,-0.070525
4,5.0,60.0,1.0,51.520406,-0.085969,51.524897,-0.067054
5,6.0,75.0,1.0,51.520406,-0.085969,51.522730,-0.064872
6,7.0,90.0,1.0,51.520406,-0.085969,51.520405,-0.064128
7,8.0,105.0,1.0,51.520406,-0.085969,51.518080,-0.064872
8,9.0,120.0,1.0,51.520406,-0.085969,51.515914,-0.067054
9,10.0,135.0,1.0,51.520406,-0.085969,51.514053,-0.070525


In [393]:
# try and combine the functions!


def data_for_map(start_latitude, start_longitude, number_points,distance):
    
    #this flag makes sure the logging table gets cleared
    flag = True
    
    

    
    concentric_circles(split=1,reset_table=0,start_latitude=start_latitude, start_longitude=start_longitude, number_points=number_points, distance=distance)
    
    max_num = (number_points+1) *distance
    
    for i in range(1,max_num):
        coordinates_df_filtered = coordinates_df_conc[coordinates_df_conc.index == i]
        distance_api(start_lat = coordinates_df_filtered.iloc[0]['start_lat'], start_lon= coordinates_df_filtered.iloc[0]['start_lon'], travel_mode = 'transit', end_lat=coordinates_df_filtered.iloc[0]['end_lat'], end_lon= coordinates_df_filtered.iloc[0]['end_lon'], rebuild_table = i)
        
        print(str(i) + ' of ' + str(max_num) +' done!')
    return (logging_table)

    
    
    

In [395]:
data_for_map(start_latitude= 51.520406,start_longitude = -0.085969,number_points= 90, distance=20)



1 of 1820 done!
2 of 1820 done!
3 of 1820 done!
4 of 1820 done!
5 of 1820 done!
6 of 1820 done!
7 of 1820 done!
8 of 1820 done!
9 of 1820 done!
10 of 1820 done!
11 of 1820 done!
12 of 1820 done!
13 of 1820 done!
14 of 1820 done!
15 of 1820 done!
16 of 1820 done!
17 of 1820 done!
18 of 1820 done!
19 of 1820 done!
20 of 1820 done!
21 of 1820 done!
22 of 1820 done!
23 of 1820 done!
24 of 1820 done!
25 of 1820 done!
26 of 1820 done!
27 of 1820 done!
28 of 1820 done!
29 of 1820 done!
30 of 1820 done!
31 of 1820 done!
32 of 1820 done!
33 of 1820 done!
34 of 1820 done!
35 of 1820 done!
36 of 1820 done!
37 of 1820 done!
38 of 1820 done!
39 of 1820 done!
40 of 1820 done!
41 of 1820 done!
42 of 1820 done!
43 of 1820 done!
44 of 1820 done!
45 of 1820 done!
46 of 1820 done!
47 of 1820 done!
48 of 1820 done!
49 of 1820 done!
50 of 1820 done!
51 of 1820 done!
52 of 1820 done!
53 of 1820 done!
54 of 1820 done!
55 of 1820 done!
56 of 1820 done!
57 of 1820 done!
58 of 1820 done!
59 of 1820 done!
60 of 

463 of 1820 done!
464 of 1820 done!
465 of 1820 done!
466 of 1820 done!
467 of 1820 done!
468 of 1820 done!
469 of 1820 done!
470 of 1820 done!
471 of 1820 done!
472 of 1820 done!
473 of 1820 done!
474 of 1820 done!
475 of 1820 done!
476 of 1820 done!
477 of 1820 done!
478 of 1820 done!
479 of 1820 done!
480 of 1820 done!
481 of 1820 done!
482 of 1820 done!
483 of 1820 done!
484 of 1820 done!
485 of 1820 done!
486 of 1820 done!
487 of 1820 done!
488 of 1820 done!
489 of 1820 done!
490 of 1820 done!
491 of 1820 done!
492 of 1820 done!
493 of 1820 done!
494 of 1820 done!
495 of 1820 done!
496 of 1820 done!
497 of 1820 done!
498 of 1820 done!
499 of 1820 done!
500 of 1820 done!
501 of 1820 done!
502 of 1820 done!
503 of 1820 done!
504 of 1820 done!
505 of 1820 done!
506 of 1820 done!
507 of 1820 done!
508 of 1820 done!
509 of 1820 done!
510 of 1820 done!
511 of 1820 done!
512 of 1820 done!
513 of 1820 done!
514 of 1820 done!
515 of 1820 done!
516 of 1820 done!
517 of 1820 done!
518 of 182

919 of 1820 done!
920 of 1820 done!
921 of 1820 done!
922 of 1820 done!
923 of 1820 done!
924 of 1820 done!
925 of 1820 done!
926 of 1820 done!
927 of 1820 done!
928 of 1820 done!
929 of 1820 done!
930 of 1820 done!
931 of 1820 done!
932 of 1820 done!
933 of 1820 done!
934 of 1820 done!
935 of 1820 done!
936 of 1820 done!
937 of 1820 done!
938 of 1820 done!
939 of 1820 done!
940 of 1820 done!
941 of 1820 done!
942 of 1820 done!
943 of 1820 done!
944 of 1820 done!
945 of 1820 done!
946 of 1820 done!
947 of 1820 done!
948 of 1820 done!
949 of 1820 done!
950 of 1820 done!
951 of 1820 done!
952 of 1820 done!
953 of 1820 done!
954 of 1820 done!
955 of 1820 done!
956 of 1820 done!
957 of 1820 done!
958 of 1820 done!
959 of 1820 done!
960 of 1820 done!
961 of 1820 done!
962 of 1820 done!
963 of 1820 done!
964 of 1820 done!
965 of 1820 done!
966 of 1820 done!
967 of 1820 done!
968 of 1820 done!
969 of 1820 done!
970 of 1820 done!
971 of 1820 done!
972 of 1820 done!
973 of 1820 done!
974 of 182

1355 of 1820 done!
1356 of 1820 done!
1357 of 1820 done!
1358 of 1820 done!
1359 of 1820 done!
1360 of 1820 done!
1361 of 1820 done!
1362 of 1820 done!
1363 of 1820 done!
1364 of 1820 done!
1365 of 1820 done!
1366 of 1820 done!
1367 of 1820 done!
1368 of 1820 done!
1369 of 1820 done!
1370 of 1820 done!
1371 of 1820 done!
1372 of 1820 done!
1373 of 1820 done!
1374 of 1820 done!
1375 of 1820 done!
1376 of 1820 done!
1377 of 1820 done!
1378 of 1820 done!
1379 of 1820 done!
1380 of 1820 done!
1381 of 1820 done!
1382 of 1820 done!
1383 of 1820 done!
1384 of 1820 done!
1385 of 1820 done!
1386 of 1820 done!
1387 of 1820 done!
1388 of 1820 done!
1389 of 1820 done!
1390 of 1820 done!
1391 of 1820 done!
1392 of 1820 done!
1393 of 1820 done!
1394 of 1820 done!
1395 of 1820 done!
1396 of 1820 done!
1397 of 1820 done!
1398 of 1820 done!
1399 of 1820 done!
1400 of 1820 done!
1401 of 1820 done!
1402 of 1820 done!
1403 of 1820 done!
1404 of 1820 done!
1405 of 1820 done!
1406 of 1820 done!
1407 of 1820

1787 of 1820 done!
1788 of 1820 done!
1789 of 1820 done!
1790 of 1820 done!
1791 of 1820 done!
1792 of 1820 done!
1793 of 1820 done!
1794 of 1820 done!
1795 of 1820 done!
1796 of 1820 done!
1797 of 1820 done!
1798 of 1820 done!
1799 of 1820 done!
1800 of 1820 done!
1801 of 1820 done!
1802 of 1820 done!
1803 of 1820 done!
1804 of 1820 done!
1805 of 1820 done!
1806 of 1820 done!
1807 of 1820 done!
1808 of 1820 done!
1809 of 1820 done!
1810 of 1820 done!
1811 of 1820 done!
1812 of 1820 done!
1813 of 1820 done!
1814 of 1820 done!
1815 of 1820 done!
1816 of 1820 done!
1817 of 1820 done!
1818 of 1820 done!
1819 of 1820 done!


,runtime,Origin,Destination,distance,travel_time,duration_value,travel_mode
0,2020-03-04 20:55:29.287371,"51.520406, -0.085969","51.529083050304436, -0.08031606526847591",1.9 km,15 mins,910,transit
1,2020-03-04 20:55:29.796766,"51.520406, -0.085969","51.52818546215703, -0.07504836879379834",1.7 km,14 mins,864,transit
2,2020-03-04 20:55:30.166232,"51.520406, -0.085969","51.526757642076156, -0.07052489542553632",1.7 km,15 mins,914,transit
3,2020-03-04 20:55:30.414895,"51.520406, -0.085969","51.52489693791481, -0.06705391235824391",1.9 km,16 mins,937,transit
4,2020-03-04 20:55:30.699588,"51.520406, -0.085969","51.52273019784939, -0.0648719612550769",2.6 km,21 mins,1284,transit
5,2020-03-04 20:55:31.007935,"51.520406, -0.085969","51.520405114023305, -0.06412773838106191",3.2 km,20 mins,1188,transit
6,2020-03-04 20:55:31.646734,"51.520406, -0.085969","51.51808014889558, -0.0648719612550769",2.6 km,19 mins,1166,transit
7,2020-03-04 20:55:31.950961,"51.520406, -0.085969","51.51591373312012, -0.0670539123582439",2.5 km,20 mins,1184,transit
8,2020-03-04 20:55:32.277066,"51.520406, -0.085969","51.51405347194712, -0.0705248954255363",1.9 km,14 mins,854,transit
9,2020-03-04 20:55:32.561023,"51.520406, -0.085969","51.5126260948546, -0.07504836879379834",1.7 km,11 mins,669,transit


In [399]:
logging_table.head(95)

,runtime,Origin,Destination,distance,travel_time,duration_value,travel_mode,lat,lon
0,2020-03-04 20:55:29.287371,"51.520406, -0.085969","51.529083050304436, -0.08031606526847591",1.9 km,15 mins,910,transit,51.529083050304436,-0.08031606526847591
1,2020-03-04 20:55:29.796766,"51.520406, -0.085969","51.52818546215703, -0.07504836879379834",1.7 km,14 mins,864,transit,51.52818546215703,-0.07504836879379834
2,2020-03-04 20:55:30.166232,"51.520406, -0.085969","51.526757642076156, -0.07052489542553632",1.7 km,15 mins,914,transit,51.526757642076156,-0.07052489542553632
3,2020-03-04 20:55:30.414895,"51.520406, -0.085969","51.52489693791481, -0.06705391235824391",1.9 km,16 mins,937,transit,51.52489693791481,-0.06705391235824391
4,2020-03-04 20:55:30.699588,"51.520406, -0.085969","51.52273019784939, -0.0648719612550769",2.6 km,21 mins,1284,transit,51.52273019784939,-0.0648719612550769
5,2020-03-04 20:55:31.007935,"51.520406, -0.085969","51.520405114023305, -0.06412773838106191",3.2 km,20 mins,1188,transit,51.520405114023305,-0.06412773838106191
6,2020-03-04 20:55:31.646734,"51.520406, -0.085969","51.51808014889558, -0.0648719612550769",2.6 km,19 mins,1166,transit,51.51808014889558,-0.0648719612550769
7,2020-03-04 20:55:31.950961,"51.520406, -0.085969","51.51591373312012, -0.0670539123582439",2.5 km,20 mins,1184,transit,51.51591373312012,-0.0670539123582439
8,2020-03-04 20:55:32.277066,"51.520406, -0.085969","51.51405347194712, -0.0705248954255363",1.9 km,14 mins,854,transit,51.51405347194712,-0.0705248954255363
9,2020-03-04 20:55:32.561023,"51.520406, -0.085969","51.5126260948546, -0.07504836879379834",1.7 km,11 mins,669,transit,51.5126260948546,-0.07504836879379834


In [396]:
logging_table.to_csv('map_example_data.csv' , index=False)

In [401]:
#found this code here
# https://towardsdatascience.com/heat-map-your-google-payments-with-python-54ed110854d

#!pip install folium

import pandas as pd 
import folium
from folium.plugins import HeatMap
import os

for_map = logging_table
for_map['lat'] = for_map['Destination'].str.split(",", n = 1, expand = True)[0]
for_map['lon'] = for_map['Destination'].str.split(",", n = 1, expand = True)[1]
for_map = for_map[['lat','lon','duration_value']]

max_amount = float(for_map['duration_value'].max())

hmap = folium.Map(location=[51.520406, -0.085969], zoom_start=7, )

hm_wide = HeatMap( list(zip(for_map.lat.values, for_map.lon.values, for_map.duration_value.values)),
                   min_opacity=0.8,
                   max_val=max_amount,
                   radius=5, blur=10, 
                   max_zoom=1, 
                 )


# Adds the heatmap element to the map
hmap.add_child(hm_wide)

# Saves the map to heatmap.hmtl
hmap.save(os.path.join('.', 'heatmap_full2.html'))

import webbrowser

url = "file:///Users/tomcronin/Documents/GA-Maps-Project/heatmap_full2.html"
webbrowser.open(url)


True

In [409]:
#example code of plotly express
# https://plot.ly/python/bubble-maps/
#!pip install plotly-express
import plotly.express as px
df = px.data.gapminder().query("year==2007")
fig = px.scatter_geo(df, locations="iso_alpha", color="continent",
                     hover_name="country", size="pop",
                     projection="natural earth")
fig.show()

NameError: name 'offline' is not defined

In [ ]:
# should try and figure out how to pass in multiple coords at once...could do so with the polyline encode?


# driving, walking, bicycling, transit   are the possible options
from urllib.request import urlretrieve
from urllib.parse import urlencode


def distance_api_bulk(start_lat, start_lon, travel_mode, end_lat, end_lon, rebuild_table):
    
    global logging_table
    global Origin
    global Destination
    
    #read in API key from local directory
    API_KEY = open('API_key.txt','r').read()
    
    Origin = str(str(start_lat) + ', ' + str(start_lon))
    Destination = str(str(end_lat) + ', ' + str(end_lon))
    
    #convert coordinates to encoded polyline to make url easier to parse
    origin_polyline = 'enc:' + polyline.encode([(start_lat, start_lon)], 5) + ':'
    destination_polyline = 'enc:' + polyline.encode([(end_lat, end_lon)], 5) + ':'
    
    
    google_url = 'https://maps.googleapis.com/maps/api/distancematrix/json?origins=' + origin_polyline + '&destinations=' + destination_polyline + '&key=' + API_KEY + '&mode=' +travel_mode 
    
    google_request = requests.get(google_url)

    google_json = google_request.json()

    
    try:
        distance_text = google_json['rows'][0]['elements'][0]['distance']['text']

        
    except:
        distance_text = 'API ERROR'
        duration_text = 'API ERROR'
        duration_value = 0
        
    else:
        distance_text = google_json['rows'][0]['elements'][0]['distance']['text']
        duration_text = google_json['rows'][0]['elements'][0]['duration']['text']

        duration_value = google_json['rows'][0]['elements'][0]['duration']['value']

   # print(Origin + ' is ' + distance_text + ' away from ' + Destination +'. It will take ' + duration_text + ' to get there by ' + travel_mode)

    result_list = [datetime.datetime.now(),Origin,Destination,distance_text,duration_text,duration_value,travel_mode]

    #initialise logging table
    
    #print('rebuild_table = ' + str(rebuild_table))
    if rebuild_table == 1:
        del logging_table
    
    try:
        logging_table
    except NameError:
        #print('nameerror flagged')
        column_labels = ['runtime','Origin','Destination','distance','travel_time','duration_value','travel_mode']
        logging_table = pd.DataFrame(columns = column_labels)
        logging_table = logging_table.append(pd.Series(result_list, index = logging_table.columns), ignore_index=True)
    else:
        #print('no nameerror')
        logging_table = logging_table.append(pd.Series(result_list, index = logging_table.columns), ignore_index=True)
      
    return (logging_table)